## Import requirements & Parameters setting

In [5]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import glob
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, LSTM, TimeDistributed, Input
from sklearn.metrics import mean_absolute_error, mean_squared_error
from utils.functions import checkDirExist, deleteDir

batch_size = 32
epochs = 300

# set model name & model directory
model_name = '20201202_dense-rmse-position2-image2-100-batchnorm-7dense-valid02'
model_dir = os.path.join(os.getcwd(), 'models/'+model_name)
log_dir = os.path.join(os.getcwd(), 'logs/'+model_name)
checkDirExist(model_dir)

# get train data (image)
train_dir = os.path.join(os.getcwd(), 'dataset/position_2/image2-100/train/')
train_list = glob.glob(train_dir + "*.jpg")

# split validation
random.Random(4).shuffle(train_list)
vaild_num = int(len(train_list)*0.2)
valid_list = train_list[:vaild_num]
train_list = train_list[vaild_num:]

# get test data (image)
test_dir = os.path.join(os.getcwd(), 'dataset/position_2/image2-100/test/')
test_list = glob.glob(test_dir + "*.jpg")

print('train: ', len(train_list))
print('valid: ', len(valid_list))
print('test: ', len(test_list))

train:  119
valid:  29
test:  20


## Dynamic allocation for GPU (optional)

In [3]:
import keras.backend.tensorflow_backend as KTF

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   # Dynamic allocation
sess = tf.Session(config=config)
KTF.set_session(sess)

## Train / Validation / Test generator definition

In [4]:
def train_generator():
    while(True):
        random.Random(4).shuffle(train_list)
        for i, image in enumerate(train_list):
            data = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
            data = cv2.resize(data, (320, 180))
            data = img_to_array(data) / 255
            label = image.split('/')[-1].split('_')[3:6]
            
            x_train = np.expand_dims(data, axis=0)
            y_train = np.expand_dims(np.array(label), axis=0)

            y_train = y_train.astype(np.float32)
            y_train[:, 0] = (y_train[:, 0]-20)/120
            y_train[:, 1] = (y_train[:, 1]-10)/60
            y_train[:, 2] = (y_train[:, 2]-10)/30

            yield x_train, y_train, [None]

In [5]:
def valid_generator():
    while(True):
        for i, image in enumerate(valid_list):
            data = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
            data = cv2.resize(data, (320, 180))
            data = img_to_array(data) / 255
            label = image.split('/')[-1].split('_')[3:6]
            
            x_valid = np.expand_dims(data, axis=0)
            y_valid = np.expand_dims(np.array(label), axis=0)

            y_valid = y_valid.astype(np.float32)
            y_valid[:, 0] = (y_valid[:, 0]-20)/120
            y_valid[:, 1] = (y_valid[:, 1]-10)/60
            y_valid[:, 2] = (y_valid[:, 2]-10)/30

            yield x_valid, y_valid, [None]

In [6]:
def test_generator():
    for i, image in enumerate(test_list):
        data = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
        data = cv2.resize(data, (320, 180))
        data = img_to_array(data) / 255
        label = image.split('/')[-1].split('_')[3:6]

        x_test = np.expand_dims(data, axis=0)
        y_test = np.expand_dims(np.array(label), axis=0)

        x_test = x_test.astype(np.float32)
        y_test = y_test.astype(np.float32)
        yield x_test, y_test, [None]

## Build model

In [7]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [8]:
model = Sequential()
model.add(Flatten(input_shape=(180, 320, 1)))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3))

model.compile(loss=root_mean_squared_error, optimizer="adam", metrics=['mae'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 57600)             230400    
_________________________________________________________________
dense (Dense)                (None, 64)                3686464   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)  

In [9]:
from tensorflow.keras.utils import plot_model
plot_file = os.path.join(model_dir, model_name+'.png')
plot_model(model, show_shapes=True, show_layer_names=True, to_file=plot_file)

In [10]:
# save training logs
model_cbk = keras.callbacks.TensorBoard(log_dir="logs/{}".format(model_name), histogram_freq=1)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')

# save best model weights
model_mckp = keras.callbacks.ModelCheckpoint(os.path.join(model_dir, model_name+".h5"), 
                                             monitor='val_loss', 
                                             save_best_only=True, 
                                             mode='min')

In [11]:
history = model.fit(train_generator(), 
                    steps_per_epoch=batch_size, 
                    epochs=epochs,
                    validation_data=valid_generator(),
                    validation_steps=len(valid_list),
                    callbacks=[model_mckp, model_cbk])

Epoch 1/300
32/32 [==============================] - 23s 704ms/step - loss: 0.6119 - mean_absolute_error: 0.5476 - val_loss: 0.5805 - val_mean_absolute_error: 0.5258
Epoch 2/300
32/32 [==============================] - 22s 679ms/step - loss: 0.5796 - mean_absolute_error: 0.5165 - val_loss: 0.5596 - val_mean_absolute_error: 0.5049
Epoch 3/300
32/32 [==============================] - 21s 667ms/step - loss: 0.5650 - mean_absolute_error: 0.5055 - val_loss: 0.5389 - val_mean_absolute_error: 0.4847
Epoch 4/300
32/32 [==============================] - 21s 671ms/step - loss: 0.5210 - mean_absolute_error: 0.4656 - val_loss: 0.5183 - val_mean_absolute_error: 0.4644
Epoch 5/300
32/32 [==============================] - 21s 666ms/step - loss: 0.5209 - mean_absolute_error: 0.4717 - val_loss: 0.4970 - val_mean_absolute_error: 0.4442
Epoch 6/300
32/32 [==============================] - 21s 667ms/step - loss: 0.4911 - mean_absolute_error: 0.4188 - val_loss: 0.4777 - val_mean_absolute_error: 0.4261
Epoc

KeyboardInterrupt: 

## Load model (optional)

In [12]:
'''
# Load model
model_dir = "models/20201124_lstm_rmse-trajectory-full-3L-32-16-8-diff-euclidean/"
model_name = "20201124_lstm_rmse-trajectory-full-3L-32-16-8-diff-euclidean-ep280-0.075.h5"
model_path = os.path.join(model_dir, model_name)
model = keras.models.load_model(model_path,
                                custom_objects={'root_mean_squared_error': root_mean_squared_error})

eval_his = model.evaluate_generator(test_generator(), steps=len(test_list), verbose=1)
'''

'\n# Load model\nmodel_dir = "models/20201124_lstm_rmse-trajectory-full-3L-32-16-8-diff-euclidean/"\nmodel_name = "20201124_lstm_rmse-trajectory-full-3L-32-16-8-diff-euclidean-ep280-0.075.h5"\nmodel_path = os.path.join(model_dir, model_name)\nmodel = keras.models.load_model(model_path,\n                                custom_objects={\'root_mean_squared_error\': root_mean_squared_error})\n\neval_his = model.evaluate_generator(test_generator(), steps=len(test_list), verbose=1)\n'

## Model evaluate

In [ ]:
# Predict
predict = model.predict_generator(test_generator(), verbose=1, steps=len(test_list))

predict[:, 0] = predict[:, 0] * 120 + 20
predict[:, 1] = predict[:, 1] * 60 + 10
predict[:, 2] = predict[:, 2] * 30 + 10

# Present as dataframe
df = pd.DataFrame()
for i, (_, y_test, _) in enumerate(test_generator()):
    df.loc[i, 'Speed'] = y_test[:, 0]
    df.loc[i, 'Angle_H'] = y_test[:, 1]
    df.loc[i, 'Angle_V'] = y_test[:, 2]
    df.loc[i, 'Predict(Speed)'] = predict[:,0][i]
    df.loc[i, 'Predict(Angle_H)'] = predict[:,1][i]
    df.loc[i, 'Predict(Angle_V)'] = predict[:,2][i]

# Adjust data type
df = df.astype({'Speed': 'int32',
                'Angle_H': 'int32',
                'Angle_V': 'int32',
                'Predict(Speed)': 'int32',
                'Predict(Angle_H)': 'int32',
                'Predict(Angle_V)': 'int32'})

# Caculate Loss
for i in df.index:
    target = df.loc[i, ['Speed', 'Angle_H', 'Angle_V']].tolist()
    target[0] = (target[0]-20)/120
    target[1] = (target[1]-10)/60
    target[2] = (target[2]-10)/30
    pred = df.loc[i, ['Predict(Speed)', 'Predict(Angle_H)', 'Predict(Angle_V)']].tolist()
    pred[0] = (pred[0]-20)/120
    pred[1] = (pred[1]-10)/60
    pred[2] = (pred[2]-10)/30
    
    mae = mean_absolute_error(target, pred)
    rmse = np.sqrt(mean_squared_error(target, pred))
    df.loc[i, 'MAE'] = np.around(mae, decimals=6)
    df.loc[i, 'RMSE'] = np.around(rmse, decimals=6)

df.sort_values("Speed", inplace=True)
df

## Save evaluate results

In [ ]:
# Caculate mean loss
last_idx = len(df)
df.loc[last_idx, 'MAE'] = df['MAE'].mean()
df.loc[last_idx, 'RMSE'] = df['RMSE'].mean()

# Save result to csv
result_dir = os.path.join(os.getcwd(), 'predictions/')
checkDirExist(result_dir)
df.to_csv(os.path.join(result_dir, model_name+'.csv'), index=False)

## Plot history

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('mae')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Convert Jupyter(.ipynb) to Python(.py) (optional)

In [17]:
# jupyter nbconvert --to script dense_regression_rmse_image.ipynb